In [4]:
#import matplotlib 
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [5]:
import numpy as np
import pandas as pd

In [6]:
# Python SQL toolkit and ORM
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [11]:
#create engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite copy")

In [12]:
#declare base using auto_map()
base = automap_base()

In [13]:
#use basae class to reflect database tables
base.prepare(engine, reflect = True)

In [14]:
#print classes mapped to base
base.classes.keys()

['measurement', 'station']

In [15]:
#assing class to variable 
measurement = base.classes.measurement
station = base.classes.station

In [16]:
#create session
session = Session(engine)

In [17]:
#create inspector and connect to engine
inspector = inspect(engine)

In [24]:
#use the inspector to print column name within measurement 
columns = inspector.get_columns('measurement')
for column in columns:
    print(column['name'], column['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [28]:
first = session.query(measurement).last()

AttributeError: 'Query' object has no attribute 'last'

In [27]:
first.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd40f8584e0>,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65.0,
 'date': '2010-01-01',
 'id': 1}